## Intro to LangChain
LangChain is a popular framework that allow users to quickly build apps and pipelines around Large Language Models. It can be used to for chatbots, Generative Question-Anwering (GQA), summarization, and much more.

The core idea of the library is that we can "chain" together different components to create more advanced use-cases around LLMs. Chains may consist of multiple components from several modules:

- Prompt templates: Prompt templates are, well, templates for different types of prompts. Like "chatbot" style templates, ELI5 question-answering, etc

- **LLMs:** Large language models like GPT-3, BLOOM, etc

- **Agents:** Agents use LLMs to decide what actions should be taken, tools like web search or calculators can be used, and all packaged into logical loop of operations.

- **Memory:** Short-term memory, long-term memory.

In [1]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00


## Using LLMs in LangChain
LangChain supports several LLM providers, like Hugging Face and OpenAI.

Let's start our exploration of LangChain by learning how to use a few of these different LLM integrations.

## **Hugging Face**
We first need to install additional prerequisite libraries:

In [2]:
!pip install -qU huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 3.5 MB/s eta 0:00:00


For Hugging Face models we need a Hugging Face Hub API token. We can find this by first getting an account at [HuggingFace.co](https://huggingface.co/) and clicking on our profile in the top-right corner > click Settings > click Access Tokens > click New Token > set Role to write > Generate > copy and paste the token below:

## Necessary Imports

In [6]:
import os
from getpass import getpass
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

In [8]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = "HF_API_KEY"

We can then generate text using a HF Hub model (we'll use google/flan-t5-x1) using the Inference API built into Hugging Face Hub.

(The default Inference API doesn't use specialized hardware and so can be slow and cannot run larger models like bigscience/bloom-560m or google/flan-t5-xxl)

In [ ]:
# initialize HF LLM
flan_t5 = HuggingFaceHub(
    repo_id="google/flan-t5-xl",
    model_kwargs={"temperature":1e-10}
)

# build prompt template for simple question-answering
template = """Question: {question}

Answer: """
prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(
    prompt=prompt,
    llm=flan_t5
)

question = "Which year did Neil Armstrong go to the moon?"

print(llm_chain.run(question))

If we'd like to ask multiple questions we can by passing a list of dictionary objects, where the dictionaries must contain the input variable set in our prompt template ("question") that is mapped to the question we'd like to ask

In [ ]:
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]
res = llm_chain.generate(qs)
res

It is a LLM, so we can try feeding in all questions at once

In [ ]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(
    template=multi_template,
    input_variables=["questions"]
)

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=flan_t5
)

qs_str = (
    "Which NFL team won the Super Bowl in the 2010 season?\n" +
    "If I am 6 ft 4 inches, how tall am I in centimeters?\n" +
    "Who was the 12th person on the moon?" +
    "How many eyes does a blade of grass have?"
)

print(llm_chain.run(qs_str))

But with this model it doesn't work too well, we'll see this approach works better with different models soon.

## **OpenAI**
Start by installing additional prerequisites:

In [15]:
!pip install -qU openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.4 MB/s eta 0:00:00


In [26]:
os.environ['OPENAI_API_KEY'] = "OPENAI_API_KEY"

In [31]:
from langchain.chat_models import ChatOpenAI

davinci = OpenAI(model_name='text-davinci-003')

In [32]:
llm_chain = LLMChain(
    prompt=prompt,
    llm=davinci
)

print(llm_chain.run(question))

 Neil Armstrong went to the moon in 1969.


In [33]:
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]
llm_chain.generate(qs)

LLMResult(generations=[[Generation(text=' The Green Bay Packers won the Super Bowl in the 2010 season.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' 193.04 centimeters', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' The 12th person to walk on the moon was Charles Duke, Jr., who was part of the Apollo 16 mission. He and mission commander John Young became the ninth and tenth people to walk on the moon on April 21, 1972.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' A blade of grass does not have any eyes.', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'total_tokens': 149, 'completion_tokens': 74, 'prompt_tokens': 75}, 'model_name': 'text-davinci-003'}, run=[RunInfo(run_id=UUID('cde5cc4f-39eb-49ef-98aa-8c646a2c0258')), RunInfo(run_id=UUID('c18c105b-4601-4ba1-af94-ff6e4d497347')), RunInfo(run_id=UUID('287f2bed-5012-4a10-bbbd-4a54597

Note that the below format doesn't feed the questions in iteratively but instead all in one chunk.

In [34]:
qs = [
    "Which NFL team won the Super Bowl in the 2010 season?",
    "If I am 6 ft 4 inches, how tall am I in centimeters?",
    "Who was the 12th person on the moon?",
    "How many eyes does a blade of grass have?"
]
print(llm_chain.run(qs))


1. The New Orleans Saints 
2. 193 centimeters
3. Harrison Schmitt 
4. None


Now we can try to answer all question in one go, as mentioned, more powerful LLMs like text-davinci-003 will be more likely to handle these more complex queries.

In [35]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(
    template=multi_template,
    input_variables=["questions"]
)

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=davinci
)

qs_str = (
    "Which NFL team won the Super Bowl in the 2010 season?\n" +
    "If I am 6 ft 4 inches, how tall am I in centimeters?\n" +
    "Who was the 12th person on the moon?" +
    "How many eyes does a blade of grass have?"
)

print(llm_chain.run(qs_str))

The Green Bay Packers won the Super Bowl in the 2010 season.
If you are 6 ft 4 inches, you are 193.04 centimeters tall.
The 12th person on the moon was Charles Duke.
A blade of grass has no eyes.
